In [ ]:
import sys
sys.path.append('../..')
from pymongo import UpdateOne
import matplotlib.pyplot as plt
from exploration.config import mongo_inst
from pymongo import MongoClient
from mlpp.data_collection.sample import osuDumpSampler

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
db = client.osu_random_db

In [ ]:
data = mongo_inst["osu_random_db"]

In [ ]:
test = osuDumpSampler(data)

In [ ]:
test.use_random_sample("sample_scores_1M", 6500) #sample of 1 million random documents 

In [ ]:
collection = data["sample_scores_1M"]

In [ ]:
"""
from sample of 1 million, take only that are more recent than 12/01/2018---leads to about 500k
"""
test.get_more_recent_than(collection, 2018, 12, 1, 0, 0, 0, "sample_scores_500k") 


In [ ]:
collection2 = data["sample_scores_500k"]

In [ ]:
"""
max est user pp in collection of 500k
"""
cursor=db.sample_scores_500k.aggregate(
   [
     {
       "$group":
         {
           "_id": {},
           "max": { "$max": "$mlpp.est_user_pp" }
         }
     }
   ]
)
for document in cursor:
    print(document)
print(document['max'])
max_pp = document['max']

In [ ]:
a = 0
b = 100

while b <= max_pp:
    db.uniform_collection2.insert_many(
        collection2.aggregate([
        {
        '$match': {
            'mlpp.est_user_pp' : {
                '$gt': a,
                '$lt': b,
            }
        }
    },
    {'$sample': {
        'size': 800
    }
}
        
])
    )
    a = b
    b += 100

In [ ]:
#once you have a uniform collection, use this to have a collection with the most 1000 popular maps:
db.uniform_collection2.aggregate([
    {
        '$group': {
            '_id': '$beatmap_id', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }, {
        '$limit': 1000
    }, {
        '$out': 'oneThousand_most_popular_maps2'
    }
])

## Objective 3: Beatmap id and count relationship

In [ ]:
collections = db.list_collection_names() 
print ("collections:", collections, "\n")

In [ ]:
g=[]
h=[]
x=db.oneThousand_most_popular_maps2
x1=x.find({},{ "_id": 0, "count": 1}) #finds only the count column in the collection
y1=x.find({},{ "_id": 1, "count": 0})

for i in x1:
    g.append(i['count'])

plt.plot(g,'ro')
plt.xlabel('index')
plt.ylabel('count')

In [ ]:

for i in y1:
    h.append(i['_id'])
h


In [ ]:
collections = db.list_collection_names() 
print ("collections:", collections, "\n")

In [ ]:
idList=[]
idList2=[]
z=db.osu_beatmaps_attribs_modZero
z1=z.find({},{ "_id": 0, "attrib_id": 1})
z2=z.find({},{ "_id": 0, "beatmap_id": 1})
for i in z1:
    idList.append(i['attrib_id'])
for i in z2:
    idList2.append(i['beatmap_id'])

In [ ]:
len(idList2)

In [ ]:
ODList=[]
StarList=[]
for i in idList:
    if idList[i]==5:
        ODList.append(idList[i])
for i in idList:
    if idList[i] ==17:
        StarList.append(idList[i])
ODList

In [ ]:
"""
max beatmap id in collection of beatmap attribs modZero
"""
o=db.osu_beatmaps_attribs_modZero
cursor1=o.aggregate(
   [
     {
       "$group":
         {
           "_id": {},
           "max": { "$max": "$beatmap_id" }
         }
     }
   ]
)
for i in cursor1:
    print(i)
print(i['max'])

In [ ]:
bg=[]
#idList2 is much longer than h so out of range 
#need to figure out how to get 
for i in h:
    for j in idList2:
        if h[i]==j[i]:
            bg.append(h[i])

In [ ]:
bg=[]
#idList2 is much longer than h so out of range 
#need to figure out how to get 
for i in h:
    temp=h[i-1]
    for j in idList2:
        if temp==idList2[j-1]:
            bg.append(h[i-1])

In [ ]:
for i in h:
    print(h[i-1])

In [ ]:
list1=[1,2,3]

In [ ]:
type(h)

In [ ]:
for i in h:
    print(h[i-1])

In [ ]:
h